In [8]:

import pandas as pd 
#import pyodbc
#import pymssql # pip install needed before import 
#import sys
import sqlalchemy as sql # note- import as sql is needed for reading the sql query 


# LUDU QC

## Dounload the data

In [9]:
creds = {'un': 'SANDAGNET\\cra' , 'pw': '' }
engine = sql.create_engine('mssql+pymssql://%s:%s@sql2014b8/WS' % (creds['un'], creds['pw']))
lu21 =pd.read_sql('SELECT [OBJECTID],[LCKey],[subParcel], [parcelID], [apn8],[lu],[du],[genOwnID],[MGRA14],[regionID] FROM [WS].[gis].[LUDU2021_DRAFT_20220301]',
engine.connect())
lu21

,OBJECTID,LCKey,subParcel,parcelID,apn8,lu,du,genOwnID,MGRA14,regionID
0,1,436067,417000,4170,45257212,9101,0,500,2921,0
1,2,711183,19591300,195913,67238013,1110,1,500,6447,0
2,3,530617,226884100,2268841,,4118,0,1,22847,0
3,4,495745,24767600,247676,53532214,1110,1,500,2388,0
4,5,552843,29833000,298330,57008007,1110,1,500,1640,0
...,...,...,...,...,...,...,...,...,...,...
837693,837694,930996,67182302,671823,32909001,9202,0,10,11326,0
837694,837695,192455,68658600,686586,21645031,7603,0,500,14123,0
837695,837696,11033,69600900,696009,12420126,1000,1,500,15387,0
837696,837697,884722,66960925,669609,32907002,8003,0,42,11340,0


# LUDU Codes To Be Analyzed
Here I am going through SANDAG's land use codes and seeing which should not have any dwelling units on them and reviewing from there. \
SANDAG's landuse codes can be found here: https://www.sandag.org/resources/maps_and_gis/gis_downloads/downloads/codes/Land_Use_Definitions.html?msclkid=2b916e35b4f711ec9945f63a2dbf1b8f

In [10]:
lu_codes = {'group_quarters':[1400, 1409], 
'hotels': [1500, 1501, 1502],
'vacant':[9101],
'transportation':[4101, 4102, 4103, 4104, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4120],
'cemetery':[6101],
'parks_open_spaces':[7600, 7601, 7603, 7604, 7605, 7606, 7607, 7609],
'water':[9201, 9202]
}

# Check for non-zero dwelling units
This function will check the dweeling units in question and spit out the LCKey's that are incorrect 

In [17]:
flags = pd.DataFrame(columns=['Land Use', 'LCKey', 'du'])

In [18]:
for land_use_name in lu_codes.keys():
    lu21_list= lu21[lu21.lu.isin(lu_codes[land_use_name])]
    df = lu21_list[lu21_list['du'] > 0]
    df['Land Use'] = land_use_name
    final_df = df[['Land Use', 'LCKey', 'du']]
    flags = pd.concat([flags, final_df])

C:\Users\cra\AppData\Local\Temp\ipykernel_10416\2286684651.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Land Use'] = land_use_name
C:\Users\cra\AppData\Local\Temp\ipykernel_10416\2286684651.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Land Use'] = land_use_name


In [19]:
flags = flags.reset_index(drop=True)

In [20]:
flags

,Land Use,LCKey,du
0,cemetery,488783,1
1,parks_open_spaces,855848,1
2,parks_open_spaces,889329,3
3,parks_open_spaces,855846,1
4,parks_open_spaces,886232,6
5,parks_open_spaces,5218,1
6,parks_open_spaces,145456,1
7,parks_open_spaces,823698,13
8,parks_open_spaces,543579,2
9,parks_open_spaces,861221,2


In [21]:
# Sending to the GitHub Repo
flags.to_excel('C:/Users/cra/OneDrive - San Diego Association of Governments/QA_Repository/2022/2022-16 LUDU/flags.xlsx')

# Sending to Sharepoint
flags.to_excel("C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-16 LUDU 2021/flags.xlsx")